In [ ]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays # 可选，用于处理稀疏性（如果数据量很大）
using FileIO
using JLD2
using Plots
using LaTeXStrings
using DataFrames, Colors
using StatsPlots   # 提供 boxplot，基于 Plots

include("Data_Generation_PP.jl")
include("Estimation_PP.jl")
# include("Evaluation_AP.jl")
# # include("Plot_Figures.jl")
# include("Implement_All_Methods_AP.jl")
# include("Models_AP.jl")

Calculate_Hyper_Param (generic function with 1 method)

In [2]:
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(dirname(grand_pa_dir), "/Data/")
if !isdir(data_dir)
    mkpath(data_dir)
end

## Functions

In [3]:
function set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)
    Params = Dict()
    Params["N"] = N
    Params["N_u"] = N_u
    Params["K"] = K
    Params["S_train"] = S_train
    Params["S_test"] = S_test
    Params["offdiag_sign"] = offdiag_sign
    Params["max_offdiag"] = max_offdiag
    Params["P_bar"] = P_bar
    Params["RO_coef_all"] = RO_coef_all
    Params["seed"] = seed
    return Params
end

set_Params (generic function with 1 method)

## Evaluate

#### Baseline Parameters

In [15]:
seed = 1;
N = 2;
N_u = 1;
K = 10;
S_train = 5000;
S_test = 1000;
P_bar = 5.0;
iterations = 10;
# RO_coef_all = [0.0];
RO_coef_all = [0.0,0.01,0.05,0.1,0.12,0.15,0.2,0.25,0.3];
Random.seed!(seed)
project_dir = "Joint_Pricing_and_Promotion/"

"Joint_Pricing_and_Promotion/"

### offdiag is mix

In [16]:
offdiag_sign = "mix";
max_offdiag = 1.0;
Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed);

In [17]:
sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
this_data_file = string(data_dir,project_dir,sub_file_name)
if !isdir(this_data_file)
    mkpath(this_data_file)
end
save(string(this_data_file, "Params.jld2"), Params);
println(this_data_file)

/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Joint_Pricing_and_Promotion/N=2_N_u=1_K=10_S_train=5000_offdiag_sign=mix_max_offdiag=1.0/


- Generate Data

In [18]:
A_true, B_true = Generate_Coef(N_u, N, max_offdiag, "zero");
P_train,PM_train,choice_train,PM_train_extend = Generate_Data(N,S_train,A_true,B_true,P_bar);

In [ ]:
A_hat,B_hat = Estimate_MNL_Para(PM_train_extend, P_train, choice_train,S_train, N);

([-0.08875297865776553 1.06589645763309 -0.19324516700398506; 0.7820001468140867 -0.2615386170797218 -0.4539513012235741], [-1.568900904367004 -0.0008082635812322503; -0.017310124177267967 -0.2854702373492177])

- Run Oracle

In [ ]:
print_flag=true
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

- Run ETO

In [ ]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

- Run RO under different uncertainty set

In [ ]:
model_name="Two_Side"
RST_RO = Dict(); Rev_RO = Dict();
for RO_coef in RO_coef_all
    println("Running RO with RO_coef = ",RO_coef)
    RST_RO_this = Run_RO(RO_coef, iterations, N, N_u, K, Input_Data,0.0,model_name,false);
    RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
    Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
end
save(string(this_data_file, "RST_RO.jld2"), RST_RO);

In [ ]:
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations];
Rev_RO = Dict();
for RO_coef in RO_coef_all
    RST_RO_this = RST_RO["RO_coef=$(RO_coef)"]
    Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations];
end

println("Mean Rev_Oracle: ", mean(Rev_Oracle));
println("Mean Rev_ETO: ", mean(Rev_ETO));
Rev_RO_Avg = [mean(Rev_RO["RO_coef=$(RO_coef)"]) for RO_coef in RO_coef_all];
println("Mean Rev_RO: ", Rev_RO_Avg);

In [ ]:
ratio = Rev_RO_Avg ./ mean(Rev_ETO)
plot(RO_coef_all, ratio, marker=:o, xlabel="RO_coef", ylabel="Rev_RO_Avg / Rev_ETO_Avg", legend=false)
hline!([1.0], linestyle=:dash, color=:red, label="y=1.0")
# savefig(plt, "Rev_RO_Avg_vs_Rev_ETO_Avg.png")  # 保存为当前目录下的PNG图片

In [ ]:
plot_boxplot(RO_coef_all[1:5],Rev_ETO,Rev_RO)

### offdiag is positive

In [ ]:
# offdiag_sign = "positive";
# Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)

In [ ]:
# sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
# this_data_file = string(data_dir,project_name,sub_file_name)
# if !isdir(this_data_file)
#     mkpath(this_data_file)
# end
# save(string(this_data_file, "Params.jld2"), Params);

In [ ]:
# Random.seed!(seed)
# Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);
# Input_Data = Calculate_Hyper_Param(RO_coef_all, iterations, N, N_u, K, Input_Data);
# save(string(this_data_file, "Input_Data.jld2"), Input_Data);

In [ ]:
# print_flag=true
# RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
# save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

In [ ]:
# RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
# println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
# save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

In [ ]:
# model_name="Two_Side"
# RST_RO = Dict(); Rev_RO = Dict();
# for RO_coef in RO_coef_all
#     println("Running RO with RO_coef = ",RO_coef)
#     RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
#     RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
#     Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
# end
# save(string(this_data_file, "RST_RO.jld2"), RST_RO);

In [ ]:
# for RO_coef in RO_coef_all
#     Rev_RO_this = Rev_RO["RO_coef=$(RO_coef)"]
#     println("Average = ",round(mean(Rev_RO_this./Rev_ETO),digits=4),", Rev_RO/Rev_ETO for RO_coef=$(RO_coef) = ",round.(Rev_RO_this./Rev_ETO,digits=4),)
# end

### offdiag is negative

In [ ]:
# offdiag_sign = "negative";
# Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)

In [ ]:
# sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
# this_data_file = string(data_dir,project_name,sub_file_name)
# if !isdir(this_data_file)
#     mkpath(this_data_file)
# end
# save(string(this_data_file, "Params.jld2"), Params);

In [ ]:
# print_flag=true
# RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
# save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

In [ ]:
# RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
# println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
# save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

In [ ]:
# model_name="Two_Side"
# RST_RO = Dict(); Rev_RO = Dict();
# for RO_coef in RO_coef_all
#     println("Running RO with RO_coef = ",RO_coef)
#     RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
#     RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
#     Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
# end
# save(string(this_data_file, "RST_RO.jld2"), RST_RO);